## Notes: 
line of thinking
documented motivation
understand what er are doing


proprocess 3 subjects and provide/document the cleaning times / bad channels, document the bad components

- Preprocessing Filtering, re-referencening, ICA
- Data cleaning: Time, channel and subjects
- ERP peak analysis Extract the study-relevant ERP peak subjectwise (e.g. one value per subject) and statistically test them. RQ: On which ERP-peaks do we find major difference between the conditions?
- Time Frequency analysis Calculate an induced time-frequency analysis of the main experimental contrast RQ: What oscillations underley our effect of interest?
- either:	- Mass Univariate Use a multiple regression of the main experimental contrast, controlling for reaction time (you need to calculate RT yourself). RQ: When/Where do we find differences between our conditions? Is there an influence of reaction time?
		OR
		- Decoding analysis Decode the main contrast of the experiment across time RQ: When is information about the conditions in our data available?

## Dataset Summary and Exploration
P300: A visual oddball experiment with a prolonged effect at 300ms.

In [1]:
"""
import libraries
"""
import mne
import ccs_eeg_utils
from mne_bids import (BIDSPath,read_raw_bids)
import matplotlib.pyplot as plt
import matplotlib
import os # make dir

from matplotlib.pyplot import figure
matplotlib.use('Agg') # silent mode but doesnt work
#%matplotlib inline

import numpy as np

In [2]:
def band_pass_filtering(raw, subject_id):
    """
    applys the firwin banpassfilter between 0.1 and 54 hz
    """
    print('filtering ...')

    # plot raw data before band-pass filtering
    raw.plot_psd(area_mode='range', tmax=10.0, average=False,xscale="linear",)
    ccs_eeg_utils.save_plot('01freq_before_filtering',bids_path)


    """
    band-pass filter between 0.1 Hz and 50 Hz, because Acunzo et al., 2012 recommended highp-pass <=1Hz and >40Hz and FIR. 0.75 (mildly) reduces P3 response amplitude
    59 to remove power line atifacts which are at 60 Hz
    """
    # apply filter
    raw_f = raw.copy().filter(0.1,54, fir_design='firwin')
    raw_f.plot_psd(area_mode='range', tmax=10.0, average=False,xscale="log")
    ccs_eeg_utils.save_plot('02freq_after_filtering',bids_path)

    return raw_f
    """
    The Filter seems to work as intended, as the power decreases at after 54 Hz.
    """

In [3]:
def plot_each_channel(num_channels, subject_id):
    """
    plot data
    """
    print('plotting each channel ...')
    
    # create dir to save plots if not existent        
    if not os.path.exists(str(str(bids_path)[:-37] + 'results/03whole_overlay/')): 
        os.makedirs(str(str(bids_path)[:-37] + 'results/03whole_overlay/'))

    
    # Extract a single channel and plot the whole timeseries.
    for channel_number in range(1,num_channels):#1

        # show filtered -> low frequencies
        fig = plt.figure()
        a1 = fig.add_axes([0,0,1,1])
        a1.plot((raw[channel_number-1] [0].T))
        a1.plot((raw_f[channel_number-1] [0].T))
        a1.legend(['raw', 'band-pass filtered'])
        plt.title('Subject: '+str(subject_id)+', channel: '+str(channel_number))
        plt.ylabel('potential [µV]')
        plt.xlabel('sample')
        #a1.set_xlim(0,25)
        #a1.set_ylim(-0.00003,0.00003)
        plt.savefig(str(str(bids_path)[:-37] + 'results/03whole_overlay/channel'+str(channel_number)+'.png'),bbox_inches='tight')
        #plt.show()
        plt.close(fig) # don't display figure
        
    # show raw -> high frequnecies
    channel_number = 1
    fig = plt.figure()
    a1 = fig.add_axes([0,0,1,1])
    a1.plot((raw[channel_number-1] [0].T))
    a1.plot((raw_f[channel_number-1] [0].T))
    a1.legend(['raw', 'band-pass filtered'])
    plt.title('Subject: '+str(subject_id)+', channel: '+str(channel_number))
    plt.ylabel('potential [µV]')
    plt.xlabel('sample')
    a1.set_xlim(0,25)
    a1.set_ylim(0.00294,0.003)
    ccs_eeg_utils.save_plot('04zoom_raw',bids_path)
    plt.close(fig) # don't display figure

    # show filtered -> high frequnecies are filtered
    fig = plt.figure()
    a1 = fig.add_axes([0,0,1,1])
    a1.plot((raw[channel_number-1] [0].T))
    a1.plot((raw_f[channel_number-1] [0].T))
    a1.legend(['raw', 'band-pass filtered'])
    plt.title('Subject: '+str(subject_id)+', channel: '+str(channel_number))
    plt.ylabel('potential [µV]')
    plt.xlabel('sample')
    a1.set_xlim(0,25)
    a1.set_ylim(-0.00003,0.00003)
    ccs_eeg_utils.save_plot('05zoom_filtered',bids_path)
    plt.close(fig) # don't display figure

In [4]:
def ICA(raw_f):      
    """
    ICA
    """
    print('running ICA ...')
    # set sampling frequency to 2 Hz as suggested on Pipeline slide to speed up ICA calculation
    # raw_f.resample(100, npad="auto") # not sure if needed

    # Generate an ICA object 
    ica = mne.preprocessing.ICA(method="fastica", max_iter=400) # didn't converge after default max_iter=200

    # fit the ICA on the data
    ica.fit(raw_f,verbose=True)

    # plot components
    ica.plot_components(picks=range(30))    
    ccs_eeg_utils.save_plot('06ICA_components', bids_path)

    # plot properties
    # create dir to save plots if not existent

    if not os.path.exists(str(str(bids_path)[:-37] + 'results/07ICA_properties/')): 
        os.makedirs(str(str(bids_path)[:-37] + 'results/07ICA_properties/'))
    for component in range(ica.n_components_):
        ica.plot_properties(raw_f,picks=component,psd_args={'fmax': 35.},reject=None)
        # save plot    
        plt.savefig(str(str(bids_path)[:-37] + 'results/07ICA_properties/component'+str(component)+'.png'),bbox_inches='tight')

        # don't display figure
        plt.close()
    
    """ TODO generalize for all subjects"""
    """
    # Artefacts of subject one
    heartbeat_artefacts = []
    blink_artefacts = [3]
    eye_artefacts = [0,1,2,6,24,29]
    muscle_artefacts = [5,7,10,13,16,18,21,23,28,29] # often non-stationary activity, small ERPs, "square-root spectrum", one electrode?, Yaw, below/behind ear, lowwe back of the head, 
    noisy_electrodes = [] # single active electrode, small ERP, strong 50/60hz line noise
    other_artefacts = []
    # brain components ideally show strong and clear ERPs, activity stationary over whole experiment, powerspectrum including alpha/beta peak at ~10hz, 
    """
    # Artefacts of subject two
    heartbeat_artefacts = []
    blink_artefacts = [0]
    eye_artefacts = []
    muscle_artefacts = [12,19,25] # often non-stationary activity, small ERPs, "square-root spectrum", one electrode?, Yaw, below/behind ear, lowwe back of the head, 
    noisy_electrodes = [29] # single active electrode, small ERP, strong 50/60hz line noise
    other_artefacts = [3,7,8,22,23]
    # brain components ideally show strong and clear ERPs, activity stationary over whole experiment, powerspectrum including alpha/beta peak at ~10hz, 

    exclude = []
    exclude.extend(heartbeat_artefacts)
    exclude.extend(blink_artefacts)
    exclude.extend(eye_artefacts)
    exclude.extend(muscle_artefacts)
    exclude.extend(other_artefacts)
    print('excluded components:', exclude)
    ica.exclude = exclude

    # add events
    evts, evts_dict_stim= add_events(raw_f)

    # before ICA
    epochs = mne.Epochs(ica.get_sources(raw_f),evts,evts_dict_stim,tmin=-0.2,tmax=0.8)
    epochs.average(picks=13).plot() # 13 for Pz?
    ccs_eeg_utils.save_plot('08Pz_before_ICA', bids_path)

    # after ICA
    epochs = mne.Epochs(raw_f,evts,evts_dict_stim,tmin=-0.2,tmax=0.8)
    epochs.average().plot(picks='Pz')
    ccs_eeg_utils.save_plot('09Pz_after_ICA', bids_path)

    # before / after overlay
    reconst_raw_f = raw_f.copy()
    ica.apply(reconst_raw_f,exclude=exclude)

    #raw_f.plot()
    #reconst_raw_f.plot()  
    ica.plot_overlay(raw_f,exclude=exclude)
    ccs_eeg_utils.save_plot('10before_after_overlay', bids_path)
    
    # save ICA 
    ica.save(str(str(bids_path)[:-4] + '_chrei_ica.fif'))

In [5]:
def add_events(raw):
    """
    add events
    """
    print('adding events ...')
    evts,evts_dict = mne.events_from_annotations(raw)

    # get all keys which contain "stimulus"
    wanted_keys = [e for e in evts_dict.keys() if "stimulus" in e]    # subset the large event-dictionairy
    
    evts_dict_stim=dict((k, evts_dict[k]) for k in wanted_keys if k in evts_dict)
    
    return evts,evts_dict_stim

In [18]:
def epoching():
    # Choosing Pz for P3 because this is the site most widely used in the literature and 
    # recommended by Kappenman et al.
    # Cut the raw data to one channel using `raw.pick_channels(["Pz"])` - 

    evts,evts_dict_stim = add_events(raw)

    raw_f.pick_channels(["Pz"])
    print(raw_f)
    plt.plot(raw_f[:,:][0].T)
    ccs_eeg_utils.save_plot('11Pz', bids_path)

    raw_f.info

    # These values reflect the values in the bids `*_events.tsv` file.
    raw_f.annotations 

    # Epoch the data 
    epochs = mne.Epochs(raw_f,evts,evts_dict_stim,tmin=-0.2,tmax=0.8,baseline=(-0.2,0)) # tmin and tmax chosen as recommended by Kappenman et al.

    # Plot all trials
    plt.plot(np.squeeze(epochs.get_data()[:,0,:].T))
    ccs_eeg_utils.save_plot('12trials', bids_path)

    return epochs,evts,evts_dict_stim

In [7]:
def erp_analysis():
    # indexing and averaging epochs
    target = epochs[["stimulus:{}{}".format(k,k) for k in [1,2,3,4,5]]].average()
    distractor = epochs[["stimulus:{}{}".format(k,j) for k in [1,2,3,4,5] for j in [1,2,3,4,5] if k!=j]].average()

    # indexing and averaging epochs
    target = epochs[["stimulus:{}{}".format(k,k) for k in [1,2,3,4,5]]].average()
    distractor = epochs[["stimulus:{}{}".format(k,j) for k in [1,2,3,4,5] for j in [1,2,3,4,5] if k!=j]].average()

    # plotting
    mne.viz.plot_compare_evokeds([target,distractor])
    ccs_eeg_utils.save_plot('13epochs_average', bids_path)
    
    return target, distractor

In [8]:
"""
pipeline

load data
read annotations
resample to 256 Hz
re-reference to P9 and P10
(Extract a single channel and plot the whole timeseries)
"""
# path where to save the datasets.
bids_root = "../local/bids"
num_subjects = 2#37
num_channels = 33

for subject in range(2,num_subjects+1):
    print('subject =',subject)
    subject_id = f"{subject:03}"

    bids_path = BIDSPath(subject=subject_id,task="P3",session="P3",
                         datatype='eeg', suffix='eeg',
                         root=bids_root)

    # read the file
    raw = read_raw_bids(bids_path)

    # fix the annotations readin
    ccs_eeg_utils.read_annotations_core(bids_path,raw)
    
    # load data
    raw.load_data()

    # delay shift
    # todo
   
    # resample / downsample frequency
    raw.resample(256, npad="auto")  # set sampling frequency to 256 Hz as in Kappenman et al. 
    
    # rerefernecing to P9 and P10 becuase Kappenman et al. says 
    # "we find that P9 and P10 provide cleaner signals than the traditional mastoid sites"
    raw.set_eeg_reference(ref_channels=['P9', 'P10'], verbose=None) 
    
    # add channel locations
    raw.set_montage('standard_1020',match_case=False)
    
    # band pass filtering between 0.1 and 54 hz
    raw_f = band_pass_filtering(raw, subject_id)
    
    
    # plot each channel
    plot_each_channel(num_channels, subject_id) # takes a while

    """TODO: remove large muscle artifacts, extreme voltage offsets, 
    or break periods longer than two seconds"""

    # ICA
    ICA(raw_f)

    epochs,evts,evts_dict_stim = epoching()
    
    target, distractor = erp_analysis()

subject = 2
Reading ../local/bids/sub-002/ses-P3/eeg/sub-002_task-P3_eeg.fdt
Reading events from ../local/bids/sub-002/ses-P3/eeg/sub-002_ses-P3_task-P3_events.tsv.
Reading channel info from ../local/bids/sub-002/ses-P3/eeg/sub-002_ses-P3_task-P3_channels.tsv.
Reading 0 ... 414719  =      0.000 ...   404.999 secs...


<ipython-input-8-f3bf0aec4b23>:24: RuntimeWarning: Did not find any coordsystem.json associated with sub-002_ses-P3_task-P3.

The search_str was "../local/bids/sub-002/**/sub-002_ses-P3*coordsystem.json"
  raw = read_raw_bids(bids_path)


EEG channel type selected for re-referencing
Applying a custom EEG reference.
filtering ...
Effective window size : 8.000 (s)
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 54 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 54.00 Hz
- Upper transition bandwidth: 13.50 Hz (-6 dB cutoff frequency: 60.75 Hz)
- Filter length: 8449 samples (33.004 sec)

Effective window size : 8.000 (s)
plotting each channel ...
running ICA ...
Fitting ICA to data using 30 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Selecting all PCA components: 30 components
Fitting ICA took 11.5s.
    Using multitaper spectrum estimation w

Transforming to ICA space (30 components)
Zeroing out 9 ICA components
Writing ICA solution to ../local/bids/sub-002/ses-P3/eeg/sub-002_ses-P3_task-P3_events_chrei_ica.fif...
adding events ...
Used Annotations descriptions: ['response:201', 'response:202', 'stimulus:11', 'stimulus:12', 'stimulus:13', 'stimulus:14', 'stimulus:15', 'stimulus:21', 'stimulus:22', 'stimulus:23', 'stimulus:24', 'stimulus:25', 'stimulus:31', 'stimulus:32', 'stimulus:33', 'stimulus:34', 'stimulus:35', 'stimulus:41', 'stimulus:42', 'stimulus:43', 'stimulus:44', 'stimulus:45', 'stimulus:51', 'stimulus:52', 'stimulus:53', 'stimulus:54', 'stimulus:55']
<RawEEGLAB | sub-002_task-P3_eeg.fdt, 1 x 103680 (405.0 s), ~841 kB, data loaded>
Not setting metadata
Not setting metadata
200 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 200 events and 257 original time points ...
0 bad epochs dropped


**Q:** What is the unit/scale of the data?
**A:** Electric potential in [µV] and time in [ms]

**T:** Have a look at `raw.info` and note down what the sampling frequency is (how many EEG-samples per second)

In [9]:
raw.info

<Info | 11 non-empty values
 bads: []
 ch_names: FP1, F3, F7, FC3, C3, C5, P3, P7, P9, PO7, PO3, O1, Oz, Pz, CPz, ...
 chs: 30 EEG, 3 EOG
 custom_ref_applied: True
 dig: 33 items (3 Cardinal, 30 EEG)
 highpass: 0.0 Hz
 line_freq: 60
 lowpass: 128.0 Hz
 meas_date: unspecified
 nchan: 33
 projs: []
 sfreq: 256.0 Hz
 subject_info: 4 items (dict)
>

sub-002_task-P3_events.tsv file:

| onset   | duration | sample | trial    | stim_file | value |
|---------|----------|--------|----------|-----------|-------|
| 11.9561 | 0.0      | 12244  | response | n/a       | 202   |
| 13.9639 | 0.0      | 14300  | response | n/a       | 202   |
| 18.3779 | 0.2      | 18820  | stimulus | n/a       | 45    |
| ...     | ...      | ...    | ...      | ...       | ...   |

In [10]:
epochs,evts,evts_dict_stim = epoching()

adding events ...
Used Annotations descriptions: ['response:201', 'response:202', 'stimulus:11', 'stimulus:12', 'stimulus:13', 'stimulus:14', 'stimulus:15', 'stimulus:21', 'stimulus:22', 'stimulus:23', 'stimulus:24', 'stimulus:25', 'stimulus:31', 'stimulus:32', 'stimulus:33', 'stimulus:34', 'stimulus:35', 'stimulus:41', 'stimulus:42', 'stimulus:43', 'stimulus:44', 'stimulus:45', 'stimulus:51', 'stimulus:52', 'stimulus:53', 'stimulus:54', 'stimulus:55']
<RawEEGLAB | sub-002_task-P3_eeg.fdt, 1 x 103680 (405.0 s), ~841 kB, data loaded>
Not setting metadata
Not setting metadata
200 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 200 events and 257 original time points ...
0 bad epochs dropped


In [11]:
# ensure all conditions have the same counts, as the ANOVA expects a fully balanced data matrix and 
# does not forgive imbalances that generously (risk of type-I error).
epochs.equalize_event_counts(evts_dict_stim) # TODO not sure if it's good that 100 epoches are droped

# Factor to down-sample the temporal dimension of the TFR computed by
# tfr_morlet.
decim = 2
frequencies = np.arange(7, 30, 3)  # define frequencies of interest
n_cycles = frequencies / frequencies[0]
zero_mean = False  # don't correct morlet wavelet to be of mean zero
# To have a true wavelet zero_mean should be True but here for illustration
# purposes it helps to spot the evoked response.

Dropped 100 epochs: 0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 18, 19, 20, 21, 22, 25, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 53, 55, 56, 58, 64, 65, 66, 67, 72, 83, 87, 92, 97, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 114, 115, 116, 117, 118, 119, 134, 137, 138, 139, 140, 142, 144, 145, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 176, 179, 180, 181, 182, 183, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199


In [12]:
from mne.time_frequency import tfr_morlet


In [13]:
import numpy as np
import matplotlib.pyplot as plt

import mne
from mne.time_frequency import tfr_morlet
from mne.stats import f_threshold_mway_rm, f_mway_rm, fdr_correction
from mne.datasets import sample

In [14]:
# Create TFR (time-frequency representations) for all conditions
epochs_power = list()
for condition in [epochs[k] for k in evts_dict_stim]:
    this_tfr = tfr_morlet(condition, frequencies, n_cycles=n_cycles,
                          decim=decim, average=False, zero_mean=zero_mean,
                          return_itc=False)
    this_tfr.apply_baseline(mode='ratio', baseline=(None, 0))
    this_power = this_tfr.data[:, 0, :, :]  # we only have one channel.
    epochs_power.append(this_power)

Loading data for 4 events and 257 original time points ...
Not setting metadata
Applying baseline correction (mode: ratio)
Loading data for 4 events and 257 original time points ...
Not setting metadata
Applying baseline correction (mode: ratio)
Loading data for 4 events and 257 original time points ...
Not setting metadata
Applying baseline correction (mode: ratio)
Loading data for 4 events and 257 original time points ...
Not setting metadata
Applying baseline correction (mode: ratio)
Loading data for 4 events and 257 original time points ...
Not setting metadata
Applying baseline correction (mode: ratio)
Loading data for 4 events and 257 original time points ...
Not setting metadata
Applying baseline correction (mode: ratio)
Loading data for 4 events and 257 original time points ...
Not setting metadata
Applying baseline correction (mode: ratio)
Loading data for 4 events and 257 original time points ...
Not setting metadata
Applying baseline correction (mode: ratio)
Loading data for

In [15]:
# Setup repeated measures ANOVA
# We will tell the ANOVA how to interpret the data matrix in terms of factors. 
# This is done via the factor levels argument which is a list of the number factor levels for each factor.
n_conditions = len(epochs.event_id)
n_replications = int(epochs.events.shape[0] / n_conditions)

factor_levels = [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]  # number of levels in each factor
effects = 'A*B'  # this is the default signature for computing all effects
# Other possible options are 'A' or 'B' for the corresponding main effects
# or 'A:B' for the interaction effect only (this notation is borrowed from the
# R formula language)
n_frequencies = len(frequencies)
times = 1e3 * epochs.times[::decim]
n_times = len(times)

In [16]:
# Now we’ll assemble the data matrix and swap axes so the trial replications are the first dimension and the conditions are the second dimension.

data = np.swapaxes(np.asarray(epochs_power), 1, 0)
# reshape last two dimensions in one mass-univariate observation-vector
data = data.reshape(n_replications, n_conditions, n_frequencies * n_times)

# so we have replications * conditions * observations:
print(data.shape)
print(n_conditions)

(4, 25, 1032)
25


In [17]:
# Now we’re ready to run our repeated measures ANOVA.

# Note. As we treat trials as subjects, the test only accounts for time locked responses,
# despite the ‘induced’ approach. For analysis for induced power at the group level averaged TRFs are required.
print('f_mway_rm')
fvals, pvals = f_mway_rm(data, factor_levels, effects=effects)

effect_labels = ['modality', 'location', 'modality by location']

# let's visualize our effects by computing f-images
for effect, sig, effect_label in zip(fvals, pvals, effect_labels):
    print('for')
    plt.figure()
    # show naive F-values in gray
    plt.imshow(effect.reshape(8, 211), cmap=plt.cm.gray, extent=[times[0],
               times[-1], frequencies[0], frequencies[-1]], aspect='auto',
               origin='lower')
    # create mask for significant Time-frequency locations
    effect = np.ma.masked_array(effect, [sig > .05])
    plt.imshow(effect.reshape(8, 211), cmap='RdBu_r', extent=[times[0],
               times[-1], frequencies[0], frequencies[-1]], aspect='auto',
               origin='lower')
    plt.colorbar()
    plt.xlabel('Time (ms)')
    plt.ylabel('Frequency (Hz)')
    plt.title(r"Time-locked response for '%s' (%s)" % (effect_label, ch_name))
    plt.show()

f_mway_rm


KeyboardInterrupt: 

## Cleaning Data

In [ ]:
"""
exercise 3 but is it necessary?
"""